In [1]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 4096*2 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-08 06:10:08 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.6: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA H100 PCIe. Num GPUs = 1. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.1 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8192. Num Sequences = 226.
Unsloth: vLLM's KV Cache can use up to 40.81 GB. Also swap space = 6 GB.
INFO 03-08 06:10:17 config.py:549] This model supports multiple tasks: {'classify', 'embed', 'reward', 

[W308 06:10:18.356582541 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 03-08 06:10:18 loader.py:1089] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 03-08 06:10:19 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-08 06:10:23 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-08 06:10:23 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-08 06:10:25 worker.py:267] Memory profiling takes 2.03 seconds
INFO 03-08 06:10:25 worker.py:267] the current vLLM instance can use total_gpu_memory (79.10GiB) x gpu_memory_utilization (0.60) = 47.14GiB
INFO 03-08 06:10:25 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 1.13GiB; the rest of the memory reserved for KV Cache is 40.27GiB.
INFO 03-08 06:10:25 executor_base.py:111] # cuda blocks: 20619, # CPU blocks: 3072
INFO 03-08 06:10:25 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 40.27x
INFO 03-08 06:10:29 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error 

Capturing CUDA graph shapes: 100%|██████████| 32/32 [00:22<00:00,  1.44it/s]

INFO 03-08 06:10:51 model_runner.py:1562] Graph capturing finished in 22 secs, took 0.83 GiB
INFO 03-08 06:10:51 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 28.72 seconds



Unsloth 2025.3.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import re
from datasets import load_dataset, Dataset
from search_module import search, get_question_answer, get_question_count
from rl_helpers import get_qa_dataset

train_dataset, test_dataset = get_qa_dataset()

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [5]:
import os
os.environ["WANDB_PROJECT"] = "bootstrap-search-rl"

In [6]:
# from UnslothGRPOTrainerTemp import UnslothGRPOConfig, _UnslothGRPOTrainer
import UnslothGRPOTrainerTemp
training_args = UnslothGRPOTrainerTemp.UnslothGRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    use_agentic_generate = True, # use agentic generation
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 101,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "full_local_training",

)

In [7]:


import rl_helpers
# importlib.reload(rl_helpers)


def agentic_generate(
        prompts:list[str],
        generate_fn,
        max_generations:int=6,
        ):
    return run_agent(generate_fn, tokenizer, prompts, max_generations)
model.agentic_generate = agentic_generate


from vllm import SamplingParams
verifier_sampling_params = SamplingParams(
    temperature = 0.1,
    top_p = 0.95,
    max_tokens = 4096,
)
def verifier_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = verifier_sampling_params,
    )


run_agent = rl_helpers.run_agent
reward_correctness = rl_helpers.build_reward_correctness_fn(verifier_generate_fn, tokenizer,)
reward_formatting = rl_helpers.reward_formatting

import UnslothGRPOTrainerTemp
trainer = UnslothGRPOTrainerTemp.UnslothGRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        reward_correctness,
        reward_formatting,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [8]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 608 | Num Epochs = 1 | Total steps = 101
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 167,772,160/4,796,452,864 (3.50% trained)


["What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", "What was the cause of the Lunar Module Pilot's dizziness during his postflight physical examination?", 'What was the maximum differential pressure in the ascent stage oxygen tank 2?', 'What was the maximum differential pressure in the ascent stage oxygen tank 2?', 'What

/root/AutoDidact/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 16/16 [00:00<00:00, 67.80it/s, est. speed input: 9817.59 toks/s, output: 135.64 toks/s]


rewards_per_func: tensor([0.2500, 0.2625], device='cuda:0')


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_correctness,rewards / reward_formatting
1,0.000000,0.512500,0.695523,177.375000,0.000000,0.250000,0.262500
2,0.000000,0.643750,0.697078,189.312500,0.000000,0.250000,0.393750
3,0.000500,0.681250,0.840199,215.500000,0.012084,0.375000,0.306250
4,0.000100,0.493750,0.660192,199.250000,0.002776,0.187500,0.306250
5,0.000300,0.518750,0.539054,213.750000,0.007726,0.125000,0.393750
6,0.000200,0.325000,0.495304,185.562500,0.003768,0.062500,0.262500
7,0.000200,0.581250,0.542687,221.187500,0.004105,0.187500,0.393750
8,0.000200,0.831250,0.824206,240.125000,0.004447,0.437500,0.393750
9,0.002000,0.518750,0.539054,200.875000,0.050854,0.125000,0.393750
10,0.001500,0.518750,0.517217,175.875000,0.036272,0.125000,0.393750


...

Processed prompts: 100%|██████████| 16/16 [00:00<00:00, 36.11it/s, est. speed input: 6772.70 toks/s, output: 117.70 toks/s]


rewards_per_func: tensor([0.3125, 0.6125], device='cuda:0')
['What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the location where the spacecraft was expected to land?', 'What was the cause of the inflight failure in tank 2?', 'What was the cause of the inflight failure in tank 2?', 'What was the cause of the inflight failure in tank 2?', 'What was the cause of the inflight failure in tank 2?', 'What was the cause of the inflight failure in tank 2?', 'What was the cause of the inflight failure in tank 2?', 'What was the cause of the inflight failure in tank 2

/root/AutoDidact/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 16/16 [00:00<00:00, 83.15it/s, est. speed input: 14475.87 toks/s, output: 166.38 toks/s]


rewards_per_func: tensor([0.0625, 0.6562], device='cuda:0')


TrainOutput(global_step=101, training_loss=0.01657797145482646, metrics={'train_runtime': 4469.3474, 'train_samples_per_second': 0.362, 'train_steps_per_second': 0.023, 'total_flos': 0.0, 'train_loss': 0.01657797145482646})

<a name="Inference"></a>
### Inference
Now let's try benchmark the model we trained!

In [9]:
from vllm import SamplingParams
import rl_helpers
sampling_params = SamplingParams(
    temperature = 0.5,
    top_p = 0.95,
    max_tokens = 4096,
)

def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
        lora_request = model.load_lora("full_local_training/checkpoint-101"), # load the trained LoRA
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts: 100%|██████████| 68/68 [00:02<00:00, 23.64it/s, est. speed input: 4671.56 toks/s, output: 108.82 toks/s] 

RESULTS:
percentage of correct answers: 0.5294117647058824


[{'messages': [{'role': 'system',
    'content': 'Cutting Knowledge Date: December 2023\nToday Date: 08 Mar 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'},
   {'role': 'user',
    'content': 'You are a research assistant, and you use the search_corpus tool to find answers to questions.\nGiven a question, answer it using by doing searches using the search_corpus tool.\nTo use the search_corpus tool, respond with a JSON for a function call with its proper arguments.\n\nYou may also reason in any message, thinking step by step about how to answer the question. Wrap your reasoning in <reasoning> and </reasoning> tags.\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_corpus",\n    "description": "Search over the knowledge corpus with a given query",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n     

In [13]:
# eval w/o lora
def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts: 100%|██████████| 68/68 [00:01<00:00, 45.90it/s, est. speed input: 7927.94 toks/s, output: 147.15 toks/s] 

RESULTS:
percentage of correct answers: 0.22058823529411764


[{'messages': [{'role': 'system',
    'content': 'Cutting Knowledge Date: December 2023\nToday Date: 08 Mar 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'},
   {'role': 'user',
    'content': 'You are a research assistant, and you use the search_corpus tool to find answers to questions.\nGiven a question, answer it using by doing searches using the search_corpus tool.\nTo use the search_corpus tool, respond with a JSON for a function call with its proper arguments.\n\nYou may also reason in any message, thinking step by step about how to answer the question. Wrap your reasoning in <reasoning> and </reasoning> tags.\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_corpus",\n    "description": "Search over the knowledge corpus with a given query",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n     